# M1 BIF - TP4
# Programmation Dynamique

### Valentin HULOT (M1 IL1, TP2)

In [1]:
# from dynamicProg import DynamicMatrix

In [25]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

# compatible python3

class DynamicMatrix:
    '''
    stores a matrix |S|x|T| (|S|+1 lines and |T|+1columns), sequences S and T and the score system (match, mismatch, gap)
    defines some global alignment functions
    '''
    
    def __init__(self, S, T, match, mismatch, gap):
        ''' defines and stores initial values'''
        
        self.S=S
        self.T=T
        self.gap=gap
        self.match=match
        self.mismatch=mismatch
        
        self.matrix = [0 for i in range(len(S)+1)]
        for i in range(len(S)+1):
            self.matrix[i] = [0 for j in range(len(T)+1)]
            
    def printMatrix(self):
        ''' prints the matrix'''
        width = 4
        vide = " "
        line = f"{vide:>{2*width}}"
        for j in range(0,len(self.T)):
            line += f"{self.T[j]:>{width}}"
        print(line)
        line = f"{vide:>{width}}"
        for j in range(0,len(self.T)+1):
            line += f"{self.matrix[0][j]:>{width}}"
        print(line)
        for i in range(1,len(self.S)+1):
            line = f"{self.S[i-1]:>{width}}"
            for j in range(0,len(self.T)+1):
                line += f"{self.matrix[i][j]:>{width}}"
            print(line)
    
    
    def score(self, a : str, b : str) -> int :
        '''
        Compare two char
        Return match score if equals
        else return mismatch score 
        '''
        if(a == b):
            return self.match
        return self.mismatch
    
    # --- GLOBAL ALIGNMENT ---
    
    def initGlobal(self):
        '''
        Initialize matrix for global alignment 
        Fill first line and column with gap values
        '''
        #First column
        for i in range(len(self.S)+1):
            self.matrix[i][0] = i*self.gap
        
        #First line
        for i in range(1,len(self.T)+1):
            self.matrix[0][i] = i*self.gap
            
    def fill(self):
        '''
        Fill matrix with Needleman-Wunsch algorithm
        
        '''
        for i in range(1,len(self.S)+1):
            for j in range(1,len(self.T)+1):
                #Top, gap
                delete = self.matrix[i-1][j] + self.gap

                #Left, gap
                insert = self.matrix[i][j-1] + self.gap
                
                #Corner top-left, match or mismath
                match = self.matrix[i-1][j-1] + self.score(self.S[i-1],self.T[j-1])
                
                self.matrix[i][j] = max(delete,insert,match)
    
    
    
    def printGlobalAln(self):
        '''
        Print one aligment with best S score
        '''
        #Init iterators
        i = len(self.S)
        j = len(self.T)
        
        #init output
        printSBuffer = ''
        printTBuffer = ''
        
        #while there's still letters in S and T
        while(i>0 and j>0):
            
            #Get values
            delete = self.matrix[i-1][j]
            insert = self.matrix[i][j-1]
            match = self.matrix[i-1][j-1]
            
            maxValue= max(delete,insert,match)
            
            #Add operation to buffer, decrement
            if(maxValue == match):
                printTBuffer = self.T[j-1] + printTBuffer
                printSBuffer = self.S[i-1] + printSBuffer
                i -= 1
                j -= 1
            
            elif(maxValue == delete):
                printTBuffer = '-' + printTBuffer
                printSBuffer = self.S[i-1] + printSBuffer
                i -= 1
            
            else:
                printTBuffer = self.T[j-1] + printTBuffer
                printSBuffer = '-' + printSBuffer
                j -= 1
        
        #if there's still letter in T
        #if(i == 0):     #err
        while(j>0):
            printTBuffer = self.T[j-1] + printTBuffer
            printSBuffer = '-' + printSBuffer
            j -= 1

        #if there's still letter in S
        #elif(j == 0):      #err
        while(i>0):
            printTBuffer = '-' + printTBuffer
            printSBuffer = self.S[i-1] + printSBuffer
            i -= 1
        
        #print results
        print(printTBuffer)
        print(printSBuffer)
        
        
        
    # --- LOCAL ALIGNEMENT ---
    def initSemiGlobal(self):
        '''
        Init the matrix for semi-global alignment.
        Fill first column and line with 0
        '''
         #First column
        for i in range(len(self.S)+1):
            self.matrix[i][0] = 0
        
        #First line
        for i in range(1,len(self.T)+1):
            self.matrix[0][i] = 0
       

    def fillSemiGlobal(self):
        '''
        Fill matrix with Smith-Waterman algorithm
        Same as fill w/ global, but maxValue cannot be < 0
        
        '''
        for i in range(1,len(self.S)+1):
            for j in range(1,len(self.T)+1):
                #Top, gap
                delete = self.matrix[i-1][j] + self.gap

                #Left, gap
                insert = self.matrix[i][j-1] + self.gap
                
                #Corner top-left, match or mismath
                match = self.matrix[i-1][j-1] + self.score(self.S[i-1],self.T[j-1])
                
                self.matrix[i][j] = max(delete,insert,match,0)

    
    def printSemiGlobalAln(self):
        '''
        Print a semi-global alignment with the best score
        Only show the aligned part between sub-seq of T, and S entirely
        Return best score and index of first position of S in T
        '''
        lastLineIndex = len(self.S)

        #find best score
        maxColIndex = 1
        #maxScore = -1    #err
        maxScore = self.matrix[lastLineIndex][maxColIndex]
        
        for j in range (2,len(self.T)+1):
            if(self.matrix[lastLineIndex][j] > maxScore):
                maxColIndex = j
                maxScore = self.matrix[lastLineIndex][j]
        
        #Code from printGlobalAln()
        i = lastLineIndex #Modified
        j = maxColIndex #Modified
        
        #init output
        printSBuffer = ''
        printTBuffer = ''
        
        
        #while there's still letters in S and T and the value is not 0
        current = self.matrix[i][j]
        
        while(current != 0):
            
            #Get values
            delete = self.matrix[i-1][j]
            insert = self.matrix[i][j-1]
            match = self.matrix[i-1][j-1]

            maxValue= max(delete,insert,match)

            #Add operation to buffer, decrement
            if(maxValue == match):
                printTBuffer = self.T[j-1] + printTBuffer
                printSBuffer = self.S[i-1] + printSBuffer
                i -= 1
                j -= 1

            elif(maxValue == delete):
                printTBuffer = '-' + printTBuffer
                printSBuffer = self.S[i-1] + printSBuffer
                i -= 1

            else:
                printTBuffer = self.T[j-1] + printTBuffer
                printSBuffer = '-' + printSBuffer
                j -= 1
                
            current = self.matrix[i][j]
            
        #block removed
        
        #print results
        startIndexAln = j
        #startIndexAln = j - 1

        #if there's still letter in S
        while(i>0):
            printTBuffer = '-' + printTBuffer
            printSBuffer = self.S[i-1] + printSBuffer
            i -= 1

        print(printTBuffer)
        print(printSBuffer)
        return (maxScore,startIndexAln)
        
            
        

In [26]:
##Test
dm = DynamicMatrix("AATGAATCAAT", "GATAG", +2, -1, -1)
dm.printMatrix()

           G   A   T   A   G
       0   0   0   0   0   0
   A   0   0   0   0   0   0
   A   0   0   0   0   0   0
   T   0   0   0   0   0   0
   G   0   0   0   0   0   0
   A   0   0   0   0   0   0
   A   0   0   0   0   0   0
   T   0   0   0   0   0   0
   C   0   0   0   0   0   0
   A   0   0   0   0   0   0
   A   0   0   0   0   0   0
   T   0   0   0   0   0   0


## 1. Préliminaires

#### Q1. Quel type d’objet python est utilisé pour stocker la matrice de programmation dynamique ? Avec quelle commande accède-t-on à la valeur de la cellule en face de la i-ème lettre de S et la j-ième de T ?

Le type d'objet pour la matrice est une liste de listes.  
La commande est matrix[i][j].

#### Q2. Écrire une méthode score qui prend en argument 2 caractères et qui renvoie le score d’un match si les deux caractères sont égaux et le score d’un mismatch sinon.


In [27]:
#Completed in dynamicProg.py
#right
dm.score('a','a')

2

In [28]:
#wrong
dm.score('a','b')

-1

## 2. Alignement global

#### Q3. Écrire une méthode initGlobal qui initialise la matrice pour l’alignement global (première ligne et première colonne).


In [29]:
dm.initGlobal()
dm.printMatrix()

           G   A   T   A   G
       0  -1  -2  -3  -4  -5
   A  -1   0   0   0   0   0
   A  -2   0   0   0   0   0
   T  -3   0   0   0   0   0
   G  -4   0   0   0   0   0
   A  -5   0   0   0   0   0
   A  -6   0   0   0   0   0
   T  -7   0   0   0   0   0
   C  -8   0   0   0   0   0
   A  -9   0   0   0   0   0
   A -10   0   0   0   0   0
   T -11   0   0   0   0   0


#### Q4. Écrire une méthode fill qui remplit la matrice selon l’algorithme de Needleman-Wunsch (formule de récurrence avec les trois cases voisines en haut et à gauche), et renvoie le score du meilleur alignement global des deux séquences.

In [30]:
#Completed in dynamicProg.py
dm.fill()
dm.printMatrix()

           G   A   T   A   G
       0  -1  -2  -3  -4  -5
   A  -1  -1   1   0  -1  -2
   A  -2  -2   1   0   2   1
   T  -3  -3   0   3   2   1
   G  -4  -1  -1   2   2   4
   A  -5  -2   1   1   4   3
   A  -6  -3   0   0   3   3
   T  -7  -4  -1   2   2   2
   C  -8  -5  -2   1   1   1
   A  -9  -6  -3   0   3   2
   A -10  -7  -4  -1   2   2
   T -11  -8  -5  -2   1   1


#### Q5. Proposer une méthode printGlobalAln qui affiche un alignement de meilleur score de S contre T comme dans l’exemple qui suit (dans cet exemple, le système de score suivant est utilisé : match=2, mismatch=-1, gap=-1).

In [31]:
dm.printGlobalAln()

---GA-T-A-G
AATGAATCAAT


# 3. Alignement semi-global

#### Q6. Implémenter la version semi-globale de l’alignement, où la séquence S doit être alignée en entier sur une sous-séquence de la séquence T. <br><br>Il faudra ajouter les méthodes initSemiGlobal et printSemiGlobalAln. <br><br>La fonction printSemiGlobalAln affichera un alignement semi-global de meilleur score, en affichant seulement la partie alignée (la sous-séquence de T et S en entier, sans représenter les gaps aux extrémités de la séquence S), et elle renverra le meilleur score ainsi que la position de début de l’alignement sur T.

In [32]:
# Test for semi-global
dm2 = DynamicMatrix("GATAG","AATGAATCAAT", +2, -1, -1)
dm2.printMatrix()

           A   A   T   G   A   A   T   C   A   A   T
       0   0   0   0   0   0   0   0   0   0   0   0
   G   0   0   0   0   0   0   0   0   0   0   0   0
   A   0   0   0   0   0   0   0   0   0   0   0   0
   T   0   0   0   0   0   0   0   0   0   0   0   0
   A   0   0   0   0   0   0   0   0   0   0   0   0
   G   0   0   0   0   0   0   0   0   0   0   0   0


In [33]:
# Init
dm2.initSemiGlobal()
dm2.printMatrix()

           A   A   T   G   A   A   T   C   A   A   T
       0   0   0   0   0   0   0   0   0   0   0   0
   G   0   0   0   0   0   0   0   0   0   0   0   0
   A   0   0   0   0   0   0   0   0   0   0   0   0
   T   0   0   0   0   0   0   0   0   0   0   0   0
   A   0   0   0   0   0   0   0   0   0   0   0   0
   G   0   0   0   0   0   0   0   0   0   0   0   0


In [34]:
# Fill
dm2.fillSemiGlobal()
dm2.printMatrix()

           A   A   T   G   A   A   T   C   A   A   T
       0   0   0   0   0   0   0   0   0   0   0   0
   G   0   0   0   0   2   1   0   0   0   0   0   0
   A   0   2   2   1   1   4   3   2   1   2   2   1
   T   0   1   1   4   3   3   3   5   4   3   2   4
   A   0   2   3   3   3   5   5   4   4   6   5   4
   G   0   1   2   2   5   4   4   4   3   5   5   4


In [35]:
# Print semi global alignment
(score,pos) = dm2.printSemiGlobalAln()
print("score :",score)
print("pos :",pos)

-AAT-G
GA-TAG
score : 5
pos : 0


# 4. Bonus : alignement semi-global, en mémoire linéaire


#### Q7. Implémenter la version semi-globale de telle façon que la complexité en mémoire est linéaire avec la taille de la séquence S (O(n) au lieu de O(n×m)). <br><br>On ne cherchera pas à afficher l’alignement, seulement le meilleur score. Pour cela, vous créerez une deuxième classe DMLinearMem avec les méthodes nécessaires.

In [15]:
class DMLinearMem():
    '''
    stores a matrix 2x|T| (2 lines and |T|+1columns), sequences S and T and the score system (match, mismatch, gap)
    defines some semi-global alignment functions, with linear memory complexity 
    '''

    def __init__(self, S, T, match, mismatch, gap):
        ''' defines and stores initial values'''
        
        self.S=S
        self.T=T
        self.gap=gap
        self.match=match
        self.mismatch=mismatch
        
        #only 2 lines
        self.matrix = [0 for i in range(2)]
        
        #Init lines with 0
        for i in range(2):
            self.matrix[i] = [0 for j in range(len(T)+1)]
    
    def printLine(self,count):
        '''
        Print one line of the matrix
        if first line, print also T
        else print only second line after
        '''
        width = 4
        
        #Print headers
        if(count== 0):

            vide = " "
            line = f"{vide:>{2*width}}"
        
            for j in range(0,len(self.T)):
                line += f"{self.T[j]:>{width}}"
            print(line)

            line = f"{vide:>{width}}"
            
            #print first line (no letter)
            for j in range(0,len(self.T)+1):
                line += f"{self.matrix[0][j]:>{width}}"
            print(line)
            
        
        #print only second line
        line = f"{self.S[count]:>{width}}"
        for j in range(0,len(self.T)+1):
            line += f"{self.matrix[1][j]:>{width}}"
        print(line)
           
            
    def score(self, a : str, b : str) -> int :
        '''
        Compare two char
        Return match score if equals
        else return mismatch score 
        '''
        if(a == b):
            return self.match
        return self.mismatch
    
    def getBestScore(self):     
        '''
        Fill the matrix but only keep 2 lines in memory
        '''
        #While last char of S not reached
        count = 0
        while(count < len(self.S)):
            #Calc second line
            for j in range(1,len(self.T)+1):
                #calc best value
                delete = self.matrix[0][j] + self.gap
                insert = self.matrix[1][j-1] + self.gap
                match = self.matrix[0][j-1] + self.score(self.S[count],self.T[j-1])
                
                self.matrix[1][j] = max(delete,insert,match,0)
            
            #Print
            self.printLine(count)
            
            #Copy second line to first
            self.matrix[0] = self.matrix[1]
                        
            #Reset second line
            self.matrix[1] = [0 for i in range(len(self.T)+1)]
            
            #Increment number of line done / letters of S
            count +=1
            
        return max(self.matrix[0])
                
            

In [16]:
dml = DMLinearMem("GATAG","AATGAATCAAT", +2, -1, -1)

In [17]:
score = dml.getBestScore()
print("score : ",score)

           A   A   T   G   A   A   T   C   A   A   T
       0   0   0   0   0   0   0   0   0   0   0   0
   G   0   0   0   0   2   1   0   0   0   0   0   0
   A   0   2   2   1   1   4   3   2   1   2   2   1
   T   0   1   1   4   3   3   3   5   4   3   2   4
   A   0   2   3   3   3   5   5   4   4   6   5   4
   G   0   1   2   2   5   4   4   4   3   5   5   4
score :  5
